In [15]:

import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from ncuts_utils_ablations import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_chunks_unite_instances2, remove_semantics, get_merge_pcds, merge_unite_gt
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import matplotlib.pyplot as plt 
import shutil
from tqdm import tqdm 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Here we define the dataset depending on kitti sequence!

In [16]:
DATASET_PATH = os.path.join('/media/cedric/Datasets2/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 7
old = False 

ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = '../../pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'test_data' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_folder_ncuts) == True:
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = '../../pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [17]:
if os.path.exists(out_folder + 'all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)

This cell can be ignored after first run as outputs are stored 

In [18]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz',
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )
        #o3d.visualization.draw_geometries([pcd_nonground_minor])


In [19]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd')
print(pcd_ground_minor)
kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz') as data :
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

PointCloud with 11580178 points.


Now we subsample the poses based on a voxel_size

In [20]:
if os.path.exists(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz') == False : 
	print(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz')
	poses, positions, \
	sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start,sequence_num=SEQUENCE_NUM,out_folder=out_folder)



In [21]:
with np.load(f'{out_folder}subsampled_data{SEQUENCE_NUM}_0.npz') as data:
	poses=data['poses']
	positions=data['positions']
	sampled_indices_local = data['sampled_indices_local']
	sampled_indices_global=data['sampled_indices_global']

Now we can split the point cloud into chunks based on a tbd chunk_size

In [22]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels, obbs = chunk_and_downsample_point_clouds(dataset,pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

 43%|████▎     | 293/681 [00:15<00:17, 22.23it/s]

In [ ]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None,semantics=False):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])
    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 and semantics == False : 
            pcd_colors[idcs] = background_color
        else :
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
    
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/255)
    return pcd_colored

In [ ]:
color_dict_normalized = {
    0: [0.0, 0.0, 0.0],
    1: [0.0, 0.0, 1.0],
    10: [0.9607843137254902, 0.5882352941176471, 0.39215686274509803],
    11: [0.9607843137254902, 0.9019607843137255, 0.39215686274509803],
    13: [0.9803921568627451, 0.3137254901960784, 0.39215686274509803],
    15: [0.5882352941176471, 0.23529411764705882, 0.11764705882352941],
    16: [1.0, 0.0, 0.0],
    18: [0.7058823529411765, 0.11764705882352941, 0.3137254901960784],
    20: [1.0, 0.0, 0.0],
    30: [0.11764705882352941, 0.11764705882352941, 1.0],
    31: [0.7843137254901961, 0.1568627450980392, 1.0],
    32: [0.35294117647058826, 0.11764705882352941, 0.5882352941176471],
    40: [1.0, 0.0, 1.0],
    44: [1.0, 0.5882352941176471, 1.0],
    48: [0.29411764705882354, 0.0, 0.29411764705882354],
    49: [0.29411764705882354, 0.0, 0.6862745098039216],
    50: [0.0, 0.7843137254901961, 1.0],
    51: [0.19607843137254902, 0.47058823529411764, 1.0],
    52: [0.0, 0.5882352941176471, 1.0],
    60: [0.6666666666666666, 1.0, 0.5882352941176471],
    70: [0.0, 0.6862745098039216, 0.0],
    71: [0.0, 0.23529411764705882, 0.5294117647058824],
    72: [0.3137254901960784, 0.9411764705882353, 0.5882352941176471],
    80: [0.5882352941176471, 0.9411764705882353, 1.0],
    81: [0.0, 0.0, 1.0],
    99: [1.0, 1.0, 0.19607843137254902],
    252: [0.9607843137254902, 0.5882352941176471, 0.39215686274509803],
    256: [1.0, 0.0, 0.0],
    253: [0.7843137254901961, 0.1568627450980392, 1.0],
    254: [0.11764705882352941, 0.11764705882352941, 1.0],
    255: [0.35294117647058826, 0.11764705882352941, 0.5882352941176471],
    257: [0.9803921568627451, 0.3137254901960784, 0.39215686274509803],
    258: [0.7058823529411765, 0.11764705882352941, 0.3137254901960784],
    259: [1.0, 0.0, 0.0]
}

reverse_color_dict = {tuple(v): k for k, v in color_dict_normalized.items()}


def color_pcd_kitti(pcd, labels):
    unique_labels = np.unique(labels)
    pcd_colors = np.zeros_like(np.asarray(pcd.points))
    for i in unique_labels:
        idcs = np.where(labels == i)
        idcs = idcs[0]
        pcd_colors[idcs] = np.array(color_dict_normalized[int(i)])

    pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
    return pcd


In [ ]:
def get_semantic_map(inst_labels, sem_labels, sem_classes=[49, 50, 51, 70, 72]):

    semantic_map = {}
    total_merges = 0

    inst_labels = inst_labels[np.isin(sem_labels, sem_classes)]
    sem_labels = sem_labels[np.isin(sem_labels, sem_classes)]
    for inst in np.unique(inst_labels)[1:]:
        intersect, area = np.unique(sem_labels[inst_labels == inst], return_counts=True)
        main_sem_label = intersect[np.argmax(area)]
        semantic_map[inst] = main_sem_label
        to_merge_sem_labels = intersect[intersect != main_sem_label]
        num_merges = len(to_merge_sem_labels)
        total_merges += num_merges
    return semantic_map


def get_num_splits(intersection, sem_classes, t_area, t_score, t_ratio):
    result = {
        "inst": [],
        "area": [],
        "score": [],
        "sem_labels": [],
        "num_splits": [],
        "sem_splits": [],
        "main_sem_label": [],
        "other_splits": [],
        "valid_sem_label": [],
        "sem-sem_splits": [],
    }
    for idx, inst in enumerate(intersection["inst"]):
        # print(f"inst: {inst}, area: {intersection['area'][inst]}, score: {intersection['score'][inst]}, sem_labels: {intersection['sem_labels'][inst]}, num_intersection: {intersection['num_intersection'][inst]}")
        area = np.sum(intersection["area"][idx])
        score = intersection["score"][idx]
        sem_label = intersection["sem_labels"][idx]
        if area > t_area and score < t_score and inst != 0:
            intersect_area = intersection["area"][idx]
            num_splits = np.count_nonzero(intersect_area / area > t_ratio) - 1
            valid_sem_label = sem_label[intersect_area / area > t_ratio]
            sem_sem_splits = np.count_nonzero(np.isin(valid_sem_label, sem_classes)) - 1
            other_splits = num_splits - sem_sem_splits
            result["inst"].append(inst)
            result["area"].append(area)
            result["score"].append(score)
            result["sem_labels"].append(sem_label)
            result["num_splits"].append(num_splits)
            result["sem_splits"].append(sem_label[intersect_area / area > t_ratio])
            result["main_sem_label"].append(sem_label[np.argmax(intersect_area)])
            result["other_splits"].append(other_splits)
            result["valid_sem_label"].append(valid_sem_label)
            result["sem-sem_splits"].append(sem_sem_splits)
        num_splits = np.sum(result["num_splits"])
        sem2sem_splits = np.sum(result["sem-sem_splits"])
        # print(f"inst: {inst}, area: {area}, score: {score}, sem_labels: {intersection['sem_labels'][idx]}, num_intersection: {intersection['num_intersection'][idx]}, num_splits: {num_splits}")
    return result, (num_splits, sem2sem_splits)


def create_sub_instances(inst_labels, sem_labels, result):
    new_inst_labels = inst_labels.copy()
    last_inst_id = np.max(inst_labels)
    for inst in np.unique(inst_labels):

        if inst in result["inst"]:
            idx = np.where(result["inst"] == inst)[0][0]
            sem_splits = result["sem_splits"][idx]
            main_sem_label = result["main_sem_label"][idx]
            to_split = sem_splits[sem_splits != main_sem_label]
            for split in to_split:
                last_inst_id += 1
                # print(f"Splitting instance {inst} with sem label {split} into {last_inst_id}")
                new_inst_labels[
                    np.logical_and(inst_labels == inst, sem_labels == split)
                ] = last_inst_id
    return new_inst_labels


def get_intersection(labels_sem, pred_labels, ignore_labels=[0, 1, 52, 99, 40, 48, 49]):
    intersection = {
        "inst": [],
        "sem_labels": [],
        "area": [],
        "num_intersection": [],
        "score": [],
    }
    for i in ignore_labels:
        labels_sem[labels_sem == i] = 0
    pred_labels = pred_labels[labels_sem != 0]
    labels_sem = labels_sem[labels_sem != 0]
    for label in np.unique(pred_labels):
        intersect, area = np.unique(
            labels_sem[pred_labels == label], return_counts=True
        )
        intersection["inst"].append(label)
        intersection["sem_labels"].append(intersect)
        intersection["area"].append(area)
        intersection["num_intersection"].append(len(intersect))
        max_area = np.max(area)
        # error_area = np.sum(area[area != max_area])
        score = max_area / np.sum(area)
        intersection["score"].append(score)
    return intersection


In [ ]:
def get_subpcd(pcd, indices, colors=False, normals=False):
    subpcd = o3d.geometry.PointCloud()
    subpcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points)[indices])
    new_cols = copy.deepcopy(np.asarray(pcd.colors)[indices])
    subpcd.colors = o3d.utility.Vector3dVector(new_cols)
    return subpcd

In [ ]:
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.1
proximity_threshold = 1.0
tarl_norm = False
ncuts_threshold = 0.005
        
out_name = 'tarl_dino_spatial' + str(SEQUENCE_NUM) 
limit = len(center_ids)
limit = 5
VISIBLE = True 
# Generate 30 different colors
 

COLORS = generate_random_colors(400)
COLORS = [(col[0]/255.,col[1]/255.,col[2]/255.) for col in COLORS]


        
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'


if os.path.exists(out_kitti_instance) == True : 
        shutil.rmtree(out_kitti_instance)
os.makedirs(out_kitti_instance)


out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'

if os.path.exists(out_kitti_semantic) == True : 
        shutil.rmtree(out_kitti_semantic)
os.makedirs(out_kitti_semantic)




patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
semantics = np.hstack((kitti_labels_orig['seg_nonground'].reshape(-1,),kitti_labels_orig['seg_ground'].reshape(-1,)))

instances = np.hstack((kitti_labels_orig['instance_nonground'].reshape(-1,),kitti_labels_orig['instance_ground'].reshape(-1,)))
print('number of instances',np.unique(instances))
merge_pcd = o3d.geometry.PointCloud()    



print("total sequence number",limit)
for sequence in tqdm(range(0,limit)):
                print('Sequence',center_ids[sequence])
                merged_chunk,file_name, pcd_chunk, pcd_chunk_ground,inliers, inliers_ground = ncuts_chunk(dataset,list(indices),pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,list(sampled_indices_global),
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold,obb=obbs[sequence],hpr_radius=20)
                
                cols, labels_kitti_cur = np.unique(
                np.asarray(pcd_chunk.colors), axis=0, return_inverse=True
                )
                
                visible_parts = np.where(labels_kitti_cur != 0)[0]
                
                #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_chunk,labels_kitti_cur)])
                #o3d.visualization.draw_geometries([get_subpcd(pcd_chunk,visible_parts)])
                
                seg_ground = kitti_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                inst_ground = kitti_labels['ground']['instance'][sequence][inliers][inliers_ground]
                
                file_name = str(center_ids[sequence]).zfill(6) + '.pcd'
                
                sem_non_ground = kitti_labels['nonground']['semantic'][sequence].reshape(-1,)
                inst_non_ground = kitti_labels['nonground']['instance'][sequence].reshape(-1,)
                if VISIBLE : 
                        sem_non_ground = sem_non_ground[visible_parts]
                        inst_non_ground = inst_non_ground[visible_parts]
                        pcd_chunk = get_subpcd(pcd_chunk,visible_parts)
                        
                cur_pred = pcd_chunk + pcd_chunk_ground
                
                kitti_chunk_instance = color_pcd_by_labels(copy.deepcopy(pcd_chunk),inst_non_ground,
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_instance_ground = color_pcd_by_labels(copy.deepcopy(pcd_chunk_ground),inst_ground.reshape(-1,),
                                        colors=colors,gt_labels=instances)

                semantics_non_ground = color_pcd_kitti(copy.deepcopy(pcd_chunk),sem_non_ground)
                semantics_ground = color_pcd_kitti(copy.deepcopy(pcd_chunk_ground),seg_ground.reshape(-1,))
                kitti_semantics = np.hstack((kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                                                
                
                o3d.io.write_point_cloud(out_folder_ncuts + file_name, cur_pred , write_ascii=False, compressed=False, print_progress=False)
                print('write gt',out_kitti_instance + file_name)
                o3d.io.write_point_cloud(out_kitti_instance + file_name, kitti_chunk_instance + kitti_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)
                o3d.io.write_point_cloud(out_kitti_semantic + file_name, semantics_non_ground + semantics_ground, write_ascii=False, compressed=False, print_progress=False)

number of instances [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131]
total sequence number -1


0it [00:00, ?it/s]


In [ ]:
out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'
point_cloud_semantis = get_merge_pcds(out_kitti_semantic)
seg_pcd = merge_unite_gt(point_cloud_semantis)
cols, labels_kitti_cur = np.unique(
                np.asarray(seg_pcd.colors), axis=0, return_inverse=True
            )

seg_labels = np.zeros(np.asarray(seg_pcd.colors).shape[0]) #semantic labels 
for label in np.unique(labels_kitti_cur):
                idcs = np.where(labels_kitti_cur == label)[0]
                col_cur = np.asarray(seg_pcd.colors)[idcs][0]
                seg_labels[idcs] = reverse_color_dict[tuple(col_cur)]

print('obtained semantic labels')
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'
out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'
point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)
instance_pcd = merge_unite_gt(point_clouds_kitti_instances)

point_clouds_pred = get_merge_pcds(out_folder_ncuts)
merge = merge_chunks_unite_instances2(point_clouds_pred)

[]


IndexError: list index out of range

In [ ]:
from collections import Counter

unique_colors, label_inst = np.unique(np.asarray(instance_pcd.colors), axis=0, return_inverse=True)
unique_colors, labels_ncuts_all = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)

new_ncuts_labels = remove_semantics(label_inst,labels_ncuts_all)
print(labels_ncuts_all.shape[0])
print(label_inst.shape[0])
metrics_ncuts = Metrics(name='ncuts')
metrics_ncuts.update_stats(labels_ncuts_all,new_ncuts_labels,label_inst)



sem_classes = [49, 50, 51, 70, 72]
ignore_labels = [0, 1, 52, 99, 40, 48, 49]

print("Computing metrics with Visibility setting : ",VISIBLE)
sem_labels = seg_labels
intersection = get_intersection(sem_labels, labels_ncuts_all)
result, (num_splits, sem2sem_splits) = get_num_splits(
            intersection, sem_classes, 200, 0.9, 0.02
        )
new_labels = create_sub_instances(labels_ncuts_all, sem_labels, result)
semantic_map = get_semantic_map(new_labels, sem_labels, sem_classes)
value_counts = Counter(semantic_map.values())
num_merges = np.sum(list(value_counts.values())) - len(value_counts)

print("Num merges",num_merges)
print("Num splits",num_splits)
print("Num actions",num_merges + num_splits)
print("Num sem2sem splits ", sem2sem_splits)

Processing: 100%|██████████| 23/23 [00:00<00:00, 896.55it/s]


838241
838241
start
Metrics for file ncuts
update stats


100%|██████████| 9/9 [00:00<00:00, 2237.23it/s]


got all the idcs
full stats


100%|██████████| 1/1 [00:00<00:00,  6.09it/s]


{'fScore': 0.888888888888889, 'precision': 1.0, 'recall': 0.8, 'panoptic': 0.7474371574787557}
S_assoc score :  0.6611740659616921
AP @ 0.25 80.0
AP @ 0.5 80.0
AP @ [0.5:0.95] 54.998
Computing metrics with Visibility setting :  True
Num merges 10
Num splits 10
Num actions 20
Num sem2sem splits  5
